In [1]:
import os
import sys 
from pathlib import Path


In [2]:
project_root = Path(__file__).parent.parent if "__file__" in locals() else Path().resolve().parent
sys.path.append(str(project_root))

In [3]:
from dataclasses import dataclass

from src.Briefly.constants import * 
from src.Briefly.utils.common import read_yaml, create_dirs

from src.Briefly.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

c:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-01-08 03:01:05,280: INFO: config: PyTorch version 2.5.1+cu118 available.]


In [4]:
CONFIG_PATH = project_root / CONFIG_PATH
PARAMS_PATH = project_root / PARAMS_PATH

In [5]:
@dataclass
class DataTransformConfig:
    root_dir: Path
    dataset_path: Path
    tokenizer: Path

Configuration:

In [6]:
class ConfigManager: 
    def __init__(self, config_path: Path = CONFIG_PATH, params_path: Path = PARAMS_PATH):
        self.config = read_yaml(config_path) 
        self.params = read_yaml(params_path)

        create_dirs([self.config.artifacts_root])
    
    def get_datatransform_config(self) -> DataTransformConfig:
        config = self.config.data_transform

        create_dirs([config.root_dir])

        data_transform_config = DataTransformConfig(
            root_dir=config.root_dir,
            dataset_path=config.dataset_path,
            tokenizer=config.tokenizer,
        )
        return data_transform_config


Components:

In [7]:
class DataTransform:
    def __init__(self, config:DataTransformConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer)
    def features_maker(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def dataset_transform(self):
        samsum_data = load_from_disk(self.config.dataset_path)
        samsum_data_pt = samsum_data.map(self.features_maker, batched=True)
        samsum_data_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

        

In [8]:
config = ConfigManager()
data_transform_config = config.get_datatransform_config()
data_transform = DataTransform(data_transform_config)
data_transform.dataset_transform()

[2025-01-08 03:01:05,541: INFO: common: YaML File: C:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\config\config.yaml loaded successfully]
[2025-01-08 03:01:05,541: INFO: common: YaML File: C:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\params.yaml loaded successfully]
[2025-01-08 03:01:05,541: INFO: common: Creating directory at: artifacts]
[2025-01-08 03:01:05,541: INFO: common: Creating directory at: artifacts/data_transform]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\venv\lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 204441.70 examples/s]
